# Sentiment Analysis with Deep Learning

## Loading Packages

In [120]:
%reset -f
import gensim
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Activation, Dense
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.layers import BatchNormalization


## Preparing Data

In [121]:
import pandas as pd
import numpy as np
#import text_normalizer as tn
#import model_evaluation_utils as meu
import nltk

# set precision for numpy array printing
np.set_printoptions(precision=2, linewidth=80)

# read in the dataset from a csv file using pandas
dataset = pd.read_csv('dataset_elec_4000.csv')

# showing the data
print(dataset.head())

# extract the 'review' and 'rating' columns as numpy arrays
reviews = np.array(dataset['review'])
sentiments = np.array(dataset['rating'])

# print the data type and shape of 'reviews' and 'sentiments'
type(reviews)
reviews.shape
sentiments.shape
# print(f'type(reviews): {type(reviews)}')
# print(f'reviews.shape: {reviews.shape}')
# print(f'sentiments.shape: {sentiments.shape}')

# build train and test datasets
train_reviews = reviews[:3500]
train_sentiments = sentiments[:3500]
test_reviews = reviews[:500]
test_sentiments = sentiments[:500]

## Processing is ignored
# create copies of the original training and test datasets
norm_train_reviews = train_reviews
norm_test_reviews = test_reviews

                                              review  rating
0  This case is just beautiful. I can't think of ...     1.0
1  My husband purchased these because he likes mo...     1.0
2  Very disappointed.  This item worked a time or...     0.0
3  ...first of all, this Lightning cable does exa...     1.0
4  Very bad, slow, flakey software. Very slow. I ...     0.0


In [122]:
print(len(train_reviews))

3500


In [123]:
from nltk.tokenize.toktok import ToktokTokenizer

# create an instance of the ToktokTokenizer
tokenizer = ToktokTokenizer()

# create an instance of the LabelEncoder and set number of class to 2
le = LabelEncoder()
num_classes=2 

# tokenize train reviews and encode train labels, fit the label encoder to the training labels and transform them to encoded labels, and convert the encoded labels to one-hot encoded labels
tokenized_train = [tokenizer.tokenize(text)
                   for text in norm_train_reviews]
y_tr = le.fit_transform(train_sentiments)
y_train = tensorflow.keras.utils.to_categorical(y_tr, num_classes)

# tokenize test reviews and encode test labels, fit the label encoder to the test labels and transform them to encoded labels, and convert the encoded labels to one-hot encoded labels
tokenized_test = [tokenizer.tokenize(text)
                   for text in norm_test_reviews]
y_ts = le.fit_transform(test_sentiments)
y_test = tensorflow.keras.utils.to_categorical(y_ts, num_classes)



# print the class label encoding map and some examples of the encoded and one-hot encoded labels
print('Sentiment class label map:', dict(zip(le.classes_, le.transform(le.classes_))))
print('Sample test label transformation:\n'+'-'*35,
      '\nActual Labels:', test_sentiments[:3], '\nEncoded Labels:', y_ts[:3], 
      '\nOne hot encoded Labels:\n', y_test[:3])

Sentiment class label map: {0.0: 0, 1.0: 1}
Sample test label transformation:
----------------------------------- 
Actual Labels: [1. 1. 0.] 
Encoded Labels: [1 1 0] 
One hot encoded Labels:
 [[0. 1.]
 [0. 1.]
 [1. 0.]]


In [124]:
print(len(norm_test_reviews))

500


## Training Word Embeddings

In [125]:
# measure the execution time of the cell
%%time

# build word2vec model (set the number of features in the word embeddings)
w2v_num_features = 512
w2v_model = gensim.models.Word2Vec(tokenized_train, 
                                   size=w2v_num_features, window=150,
                                   min_count=10, sample=1e-3, workers=16)    


CPU times: user 10.6 s, sys: 56.7 ms, total: 10.7 s
Wall time: 5.93 s


In [126]:
## This model uses the document word vector averaging scheme
## Use the average word vector representations to represent one document

# Transforms a list of documents (a corpus) into a feature matrix
def averaged_word2vec_vectorizer(corpus, model, num_features):

    # create a set of words in the model's vocabulary
    vocabulary = set(model.wv.index2word)
    
    # Calculates the average of the word vectors of the words in a list
    def average_word_vectors(words, model, vocabulary, num_features):

        # initialize a zero vector of size num_features
        feature_vector = np.zeros((num_features,), dtype="float64")

        # initialize a counter
        nwords = 0.
        
        # if the word is in the vocabulary, add its word vector to the feature vector and increment the counter
        for word in words:
            if word in vocabulary: 
                nwords = nwords + 1.
                feature_vector = np.add(feature_vector, model.wv[word])

        # if the counter is non-zero, divide the feature vector by the counter to get the average
        if nwords:
            feature_vector = np.divide(feature_vector, nwords)

        return feature_vector

    # apply the average_word_vectors function to each document in the corpus to get a list of feature vectors representing the documents
    features = [average_word_vectors(tokenized_sentence, model, vocabulary, num_features)
                    for tokenized_sentence in corpus]
    
    # return the feature matrix as a numpy array
    return np.array(features)

In [127]:
# generate feature matrices for the training and test datasets
avg_wv_train_features = averaged_word2vec_vectorizer(corpus=tokenized_train, model=w2v_model,
                                                     num_features=w2v_num_features)
avg_wv_test_features = averaged_word2vec_vectorizer(corpus=tokenized_test, model=w2v_model,
                                                    num_features=w2v_num_features)

## Building Model

In [128]:
def construct_deepnn_architecture(num_input_features):
    # create a Sequential model
    dnn_model = Sequential()

    # add the first hidden layer
    dnn_model.add(Dense(512, input_shape=(num_input_features,), kernel_initializer='glorot_uniform'))
    dnn_model.add(BatchNormalization()) # normalize the activations of the previous layer (improve stability of the network)
    dnn_model.add(Activation('relu'))  # apply the ReLU activation function (to present vanishing gradient problem)
    dnn_model.add(Dropout(0.2)) # apply dropout to the activations of the previous layer (prevents overfitting)
    
    # add the second hidden layer
    dnn_model.add(Dense(512, kernel_initializer='glorot_uniform'))
    dnn_model.add(BatchNormalization())
    dnn_model.add(Activation('relu'))
    dnn_model.add(Dropout(0.2))
    
    # add the third hidden layer
    dnn_model.add(Dense(512, kernel_initializer='glorot_uniform'))
    dnn_model.add(BatchNormalization())
    dnn_model.add(Activation('relu'))
    dnn_model.add(Dropout(0.2))
    
    # add the output layer
    dnn_model.add(Dense(2))  # 2 units because there are 2 classes
    dnn_model.add(Activation('softmax'))  # apply the softmax activation function

    # compile the model with the specified loss function, optimizer, and metrics
    dnn_model.compile(loss='categorical_crossentropy', optimizer='adam',                 
                      metrics=['accuracy'])
    return dnn_model

In [129]:
# create a deep neural network model with the specified number of input features
w2v_dnn = construct_deepnn_architecture(num_input_features=w2v_num_features)

## Model Fitting

### Fitting using self-trained word embeddings

In [131]:
# number of training samples to use in each iteration of training
batch_size = 100

# train the deep neural network model on the training data
w2v_dnn.fit(avg_wv_train_features, y_train, epochs=200, batch_size=batch_size, 
            shuffle=True, validation_split=0.1, verbose=1)

Epoch 1/200
32/32 [==============================] - 2s 29ms/step - loss: 0.7599 - accuracy: 0.6194 - val_loss: 0.6774 - val_accuracy: 0.6114
Epoch 2/200
32/32 [==============================] - 1s 23ms/step - loss: 0.6934 - accuracy: 0.6486 - val_loss: 0.6645 - val_accuracy: 0.5857
Epoch 3/200
32/32 [==============================] - 1s 23ms/step - loss: 0.6580 - accuracy: 0.6613 - val_loss: 0.6639 - val_accuracy: 0.6171
Epoch 4/200
32/32 [==============================] - 1s 23ms/step - loss: 0.6280 - accuracy: 0.6730 - val_loss: 0.6638 - val_accuracy: 0.5600
Epoch 5/200
32/32 [==============================] - 1s 22ms/step - loss: 0.6101 - accuracy: 0.6794 - val_loss: 0.6920 - val_accuracy: 0.5229
Epoch 6/200
32/32 [==============================] - 1s 23ms/step - loss: 0.6140 - accuracy: 0.6819 - val_loss: 0.6380 - val_accuracy: 0.6343
Epoch 7/200
32/32 [==============================] - 1s 22ms/step - loss: 0.6124 - accuracy: 0.6822 - val_loss: 0.6484 - val_accuracy: 0.5857
Epoch 

In [132]:
# use the deep neural network model to generate class predictions for the test data
y_pred = w2v_dnn.predict_classes(avg_wv_test_features)

# transform the encoded class predictions back into their original class labels
predictions = le.inverse_transform(y_pred) 

/usr/local/lib/python3.8/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [133]:
from sklearn import metrics

# calculates and prints the accuracy, precision, recall, and F1 score of a model
def get_metrics(true_labels, predicted_labels):
    
    print('Accuracy:', np.round(
                        metrics.accuracy_score(true_labels, 
                                               predicted_labels),
                        4))
    print('Precision:', np.round(
                        metrics.precision_score(true_labels, 
                                               predicted_labels,
                                               average='weighted'),
                        4))
    print('Recall:', np.round(
                        metrics.recall_score(true_labels, 
                                               predicted_labels,
                                               average='weighted'),
                        4))
    print('F1 Score:', np.round(
                        metrics.f1_score(true_labels, 
                                               predicted_labels,
                                               average='weighted'),
                        4))

# calculates and prints a confusion matrix for a model
def display_confusion_matrix(true_labels, predicted_labels, classes=[1,0]):
    
    total_classes = len(classes)
    level_labels = [total_classes*[0], list(range(total_classes))]

    cm = metrics.confusion_matrix(y_true=true_labels, y_pred=predicted_labels, 
                                  labels=classes)
    cm_frame = pd.DataFrame(data=cm, 
                            columns=pd.MultiIndex(levels=[['Predicted:'], classes], 
                                                  codes=level_labels), 
                            index=pd.MultiIndex(levels=[['Actual:'], classes], 
                                                codes=level_labels)) 
    print(cm_frame) 

# calculates and prints a classification report for a model
def display_classification_report(true_labels, predicted_labels, classes=[1,0]):

    report = metrics.classification_report(y_true=true_labels, 
                                           y_pred=predicted_labels, 
                                           labels=classes) 
    print(report)
    
    

# calculates and prints a summary of the model's performance. This includes the accuracy, precision, recall, F1 score, classification report, and confusion matrix
def display_model_performance_metrics(true_labels, predicted_labels, classes=[1,0]):
    print('Deep Learning DNN Model Performance metrics:')
    print('-'*30)
    get_metrics(true_labels=true_labels, predicted_labels=predicted_labels)
    print('\nDeep Learning DNN Model Classification report:')
    print('-'*30)
    display_classification_report(true_labels=true_labels, predicted_labels=predicted_labels, classes=classes)
    print('\nDeep Learning DNN Prediction Confusion Matrix:')
    print('-'*30)
    display_confusion_matrix(true_labels=true_labels, predicted_labels=predicted_labels, classes=classes)

In [134]:
# display model performance metrics using the test set
display_model_performance_metrics(true_labels=test_sentiments, predicted_labels=predictions, 
                                      classes=[1, 0])  

Deep Learning DNN Model Performance metrics:
------------------------------
Accuracy: 0.862
Precision: 0.8633
Recall: 0.862
F1 Score: 0.8619

Deep Learning DNN Model Classification report:
------------------------------
              precision    recall  f1-score   support

           1       0.89      0.83      0.86       251
           0       0.84      0.89      0.87       249

    accuracy                           0.86       500
   macro avg       0.86      0.86      0.86       500
weighted avg       0.86      0.86      0.86       500


Deep Learning DNN Prediction Confusion Matrix:
------------------------------
          Predicted:     
                   1    0
Actual: 1        209   42
        0         27  222
